## Dependencies

In [2]:
!pip install clean-text
!pip install inflect


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import re
import inflect
from cleantext import clean
import json
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import random
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
datatype = 'test' 

path = (
    'PIZZA_train.json' if datatype == 'train' else
    'PIZZA_dev.json' if datatype == 'dev' else
    'PIZZA_test.json' if datatype == 'test' else
    'unknown.json'  # Optional fallback
)

lines = (
    50000 if datatype == 'train' else
    847 if datatype == 'dev' else
    4000 if datatype == 'test' else
    0
)

print(f"Path: {path}")
print(f"Lines: {lines}")


Path: PIZZA_test.json
Lines: 4000


## Parsing JSON File

In [39]:

#! load data given file path and number of line to load
def load_data(file_path,num_lines):
    data = []
    with open(file_path, 'r') as file: 
        for _ in range(num_lines):
            data.append(json.loads(file.readline()))
    return data
def load_random_data(file_path, num_lines, last_lines):
    data = []
    
    with open(file_path, 'r') as file:
        file.seek(0, 2)
        file_size = file.tell()
        
        file.seek(max(file_size - 1024 * last_lines, 0))  
        lines = file.readlines()
        
        last_lines_data = []
        for line in lines[-last_lines:]:
            if line.strip():
                try:
                    last_lines_data.append(json.loads(line.strip()))
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON line: {line.strip()} - {e}")
        
        data.extend(last_lines_data)

        remaining_lines_size = file_size - sum(len(line) for line in lines[-last_lines:])

        for _ in range(num_lines):
            while True:
                try:
                    random_pos = random.randint(0, remaining_lines_size - 1)
                    file.seek(random_pos)
                    
                    file.readline() 
                    line = file.readline() 

                    if line.strip(): 
                        data.append(json.loads(line.strip()))
                        break 
                except json.JSONDecodeError as e:
                    continue  
                except Exception as e:
                    print(f"Error during random data load: {e}")
                    break
    
    return data


In [40]:
data=load_random_data(path,lines,1000) if datatype=='train' else load_data(path,lines)
print(data[:3])

[{'dev.SRC': 'i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage', 'dev.EXR': '(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )', 'dev.TOP': '(ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )', 'dev.PCFG_ERR': 'False'}, {'dev.SRC': 'five medium pizzas with tomatoes and ham', 'dev.EXR': '(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE MED

In [41]:
#! split json data
def getextensions(datatype):
    if datatype=='train':
        return 'train.SRC','train.EXR','train.TOP'
    elif datatype=='dev':
        return 'dev.SRC','dev.EXR','dev.TOP'
    else:
        return 'test.SRC','test.EXR','test.TOP'
def get_training_data(data,datatype='train'):
    values=getextensions(datatype)
    training_data = []
    training_exr=[]
    training_top=[]
    for item in data:
        training_data.append(item[values[0]])  
        training_exr.append(item[values[1]])  
        training_top.append(item[values[2]]) 
    return training_data,training_exr,training_top
training_data,training_exr,training_top=get_training_data(data,datatype)   
print(training_data[:2])
print(training_exr[:2])
print(training_top[:2])


['i want to order two medium pizzas with sausage and black olives and two medium pizzas with pepperoni and extra cheese and three large pizzas with pepperoni and sausage', 'five medium pizzas with tomatoes and ham']
['(ORDER (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (COMPLEX_TOPPING (QUANTITY EXTRA ) (TOPPING CHEESE ) ) (TOPPING PEPPERONI ) ) (PIZZAORDER (NUMBER 2 ) (SIZE MEDIUM ) (TOPPING OLIVES ) (TOPPING SAUSAGE ) ) (PIZZAORDER (NUMBER 3 ) (SIZE LARGE ) (TOPPING PEPPERONI ) (TOPPING SAUSAGE ) ) )', '(ORDER (PIZZAORDER (NUMBER 5 ) (SIZE MEDIUM ) (TOPPING HAM ) (TOPPING TOMATOES ) ) )']
['(ORDER i want to order (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING sausage ) and (TOPPING black olives ) ) and (PIZZAORDER (NUMBER two ) (SIZE medium ) pizzas with (TOPPING pepperoni ) and (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) ) and (PIZZAORDER (NUMBER three ) (SIZE large ) pizzas with (TOPPING pepperoni ) and (TOPPING sausage ) ) )', '(ORDER (PIZZAORDER (NUMBER five )

In [7]:
p = inflect.engine()

def replace_numbers_with_words(text):
    number_pattern = r'\b\d+\b'
    
    def number_to_words(match):
        number = int(match.group())
        return p.number_to_words(number, andword="").replace('-', '_')

    
    text = re.sub(number_pattern, number_to_words, text)
    
    # Replace standalone 'a' or 'an' with 'one' where appropriate
    text = re.sub(
        r'\b(a|an)\b(?! ((lot)|(little)|(few)|(number)|(couple)|(bit)|(load)|(stack)|(bunch)|(group)|(set)|(series)|(variety)|(range)|(amount)|(sum)|(total)))',
       'one',
        text,
        flags=re.IGNORECASE
    )
    return text

# Example usage
print(replace_numbers_with_words("place an order for a meduim pizzas and 3 cokes"))


place an order for a meduim pizzas and three cokes


In [8]:
#! a function to handle negations
def handle_negations(text):
    negations_pattern = r"\b(?:no|not|without)\s+.*?\b(?=(?:[^\w\s]|$))"
    # print(re.findall(negations_pattern, text))
    text = re.sub(negations_pattern, lambda x: ' '.join([f'not_{word}' for word in x.group(0).split()]), text)
    return text

In [9]:
def lem_word(word):
    if(word=='bit'):
        return word
    possible_pos = [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]
    lemmatizer = WordNetLemmatizer()
    for pos in possible_pos:
        word=lemmatizer.lemmatize(word,pos)
    return word


In [10]:
lemmatizer = WordNetLemmatizer()
#! stopwords list (adding not_)
stopwords = set(stopwords.words('english'))
stopwords.add('like')
not_stopwords = ['not_' + word for word in stopwords]
# stopwords.update(not_stopwords)
# stopwords.discard('a')
# stopwords.discard('an')
# stopwords.discard('not')
# stopwords.discard('no')
# stopwords.discard('can')
# stopwords.discard('not_a')
# stopwords.discard('not_an')
# stopwords.discard('not_can')
# stopwords.discard('not_no')
stopwords=set()
# stopwords.add('and')
# stopwords.add('also')

In [11]:
# import pandas as pd

# # Read the CSV file
# data = pd.read_csv('test_set.csv')

# # Access the 'order' column
# orders = data['order']

In [12]:
#! tokenize training data and remove stop words
def preprocess_training_data(training_data, stopwords):
    training_data=[replace_numbers_with_words(order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[word.lower() for word in order if word.lower() not in stopwords] for order in training_data]
    training_data = [clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True) for order in training_data]
    #! remove d letter most probably garbage
    training_data = [re.sub(r'\bd\s+', '', order) for order in training_data]
    training_data = [word_tokenize(order) for order in training_data]
    training_data = [[lem_word(word) for word in order] for order in training_data]
    return training_data
training_data = preprocess_training_data(training_data, stopwords)
print(training_data[:2])

In [15]:
#! save processed training dataset
path = (
    'trian_data_order_category.txt' if datatype == 'train' else
    'dev_data_order_category.txt' if datatype == 'dev' else
    'test_data_order_category2.txt'
)
with open(path, 'a') as f:
    for item in training_data:
        f.write("%s\n" % item)

In [49]:

def remove_unmatched_parentheses(input_string):
    result = list(input_string) 
    last_bracket_index=-1
    for i, char in enumerate(result):
        if char == ')' and i+2 < len(result):
            result[i] = ''  
            last_bracket_index=i
        elif char == '(':
            if last_bracket_index!=-1:
                result[last_bracket_index] = ')'
                last_bracket_index=-1
        elif char == ')' and i+2 >= len(result):
            result[i] = ''
            result[last_bracket_index] = ')'
    return ''.join(result)

In [50]:
#! get PIZZAORDER, DRINKORDER, NONE Labels 0=>PIZZAORDER, 1=>DRINKORDER, 2=>NONE

def get_order_category_labels(training_top, training_data, stopwords):
    # print(training_data)
    order_category_labels = []
    for i, item in enumerate(training_top):
        order_category_labels.append([2] * len(training_data[i]))
        #! remove any extra labelling
        item = re.sub(r'\)[^()]*\(', ') (', item)
        unwanted_keywords = r"\b(ORDER|SIZE|STYLE|TOPPING|COMPLEX_TOPPING|QUANTITY|NOT|NUMBER|DRINKTYPE|CONTAINERTYPE|VOLUME)\b"
        cleaned_string = re.sub('\('+unwanted_keywords, "", item)
        cleaned_string = [word for word in cleaned_string.split() if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_string)
        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        order_regex = r"\((?:PIZZAORDER|DRINKORDER).*?\)"
        extracted_orders = re.findall(order_regex, cleaned_string)
        k = 0
        for order in extracted_orders:
            order=replace_numbers_with_words(order)
            order = re.sub(r"[\(\)]", "", order)
            order=word_tokenize(order) #! fix id and don't bugs
            order=clean(order, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            order=re.sub(r'\bd\s+', '', order) #! for d removal
            tokens = word_tokenize(order)
            tokens = [lem_word(word) for word in tokens]
            j = 0
            to_index_train=training_data[i][k:]
            if 'pizzaorder' in tokens:
                tokens.remove('pizzaorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 0
                        j += 1
                    k += 1
                 #! EOP label
                # order_category_labels[i][k-1] = 3
            elif 'drinkorder' in tokens:
                tokens.remove('drinkorder')
                for word in to_index_train:
                    if j == len(tokens):
                        break
                    if word == tokens[j]:
                        order_category_labels[i][k] = 1
                        j += 1
                    k += 1
                #! EOD label
                # order_category_labels[i][k-1] = 4
    return order_category_labels

order_category_labels = get_order_category_labels(training_top, training_data, stopwords)

In [51]:
path = (
    'train_labels_order_category.txt' if datatype == 'train' else
    'dev_labels_order_category.txt'
)
for labels in order_category_labels:
    with open(path, 'a') as f: # dev
        f.write("%s\n" % labels)

In [52]:
def remove_unmatched_parentheses(text):
    stack = []
    result = list(text)
    for i, char in enumerate(text):
        if char == '(':
            stack.append(i)
        elif char == ')':
            if stack:
                stack.pop()
            else:
                result[i] = ''  
    for i in stack:
        result[i] = ''
    return ''.join(result)

def process_training_top(training_top, stopwords):
    labeled_data = []
    for i, item in enumerate(training_top):
        sentence_tokens = []
        labels = []
        unwanted_keywords = r'\(\b(ORDER|PIZZAORDER|DRINKORDER)\b'
        cleaned_string = re.sub(unwanted_keywords, "", item)
        words = cleaned_string.split()
        cleaned_words = [word for word in words if word.lower() not in stopwords]
        cleaned_string = ' '.join(cleaned_words)

        cleaned_string = remove_unmatched_parentheses(cleaned_string)
        # Regex to match nested parentheses excluding COMPLEX_TOPPING
        pattern_regex = r"\((?!COMPLEX_TOPPING\b)(\w+)\s+((?:[^\(\)]|\([^()]*\))*)\)"
        matches = list(re.finditer(pattern_regex, cleaned_string))
        for _, match in enumerate(matches):
            key, value = match.groups()
            # Handle NOT block
            if key == "NOT":
                nested_match = re.match(pattern_regex, value.strip())
                if nested_match:
                    nested_key, nested_value = nested_match.groups()
                    nested_value=replace_numbers_with_words(nested_value)
                    nested_value = clean(nested_value, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
                    nested_value = re.sub(r'\bd\s+', '', nested_value)  # Remove 'd'
                    value_tokens = word_tokenize(nested_value.strip())
                    value_tokens = [lem_word(word) for word in value_tokens]
                    sentence_tokens.extend(value_tokens)
                    labels.extend([f"NOT_{nested_key}"] * len(value_tokens))
                continue

            value = re.sub(r'\(\s*\w+\s*', '', value)  # Match '(WORD)' for complex toppings
            value = re.sub(r'[()]', '', value)  # Remove any remaining parentheses
            value_tokens = replace_numbers_with_words(value.strip())
            value_tokens = clean(value_tokens, no_line_breaks=True, no_punct=True, no_currency_symbols=True)
            value_tokens = re.sub(r'\bd\s+', '', value_tokens) 
            value_tokens = word_tokenize(value_tokens)
            value_tokens = [lem_word(word) for word in value_tokens]
            sentence_tokens.extend(value_tokens)

            label_mapping = {
                "STYLE": "STYLE",
                "SIZE": "SIZE",
                "TOPPING": "Topping",
                "NUMBER": "Number",
                "QUANTITY": "Complex-Topping",
                "DRINKTYPE": "Drinktype",
                "CONTAINERTYPE": "ContainerType",
                "VOLUME":"Volume"
            }
            labels.extend([label_mapping.get(key, "None")] * len(value_tokens))
        if sentence_tokens:
            labeled_data.append((sentence_tokens, labels))

    return labeled_data

def update_training_labels(training_data, training_labels, labeled_data,training_top):
    label_mapping = {
    "None": 0,
    "Number": 1,
    "SIZE": 2,
    "STYLE": 3,
    "Topping": 4,
    "Complex-Topping": 5,
    "NOT_TOPPING": 6,
    "NOT_STYLE": 7,
    "Drinktype": 8,
    "ContainerType": 9,
    "Volume": 10,
    }
    new_training_data = [list(tokens) for tokens in training_data]
    new_training_labels = [[0 for _ in labels] for labels in training_labels]
    for idx, (train_tokens, train_labels) in enumerate(zip(new_training_data, new_training_labels)):
        labeled_tokens, labeled_labels = labeled_data[idx] 
        train_ptr = 0
        labeled_ptr = 0
        
        while train_ptr < len(train_tokens) and labeled_ptr < len(labeled_tokens):
            if train_tokens[train_ptr] == labeled_tokens[labeled_ptr]:
                train_labels[train_ptr] = label_mapping.get(labeled_labels[labeled_ptr], label_mapping["None"])
                labeled_ptr += 1  # Move labeled_tokens pointer
            train_ptr += 1  # Move training_tokens pointer
        
        if labeled_ptr != len(labeled_tokens):
            print(f"Warning: Not all labeled tokens found in training tokens for index {idx}.")
            print(labeled_tokens)
            print(train_tokens)
            print(training_top[idx])
    return new_training_data, new_training_labels
labeled_data = process_training_top(training_top, stopwords)
Model2_data, Model2_labels = update_training_labels(training_data, order_category_labels, labeled_data,training_top)
print(training_data[168:170])
print(order_category_labels[168:170]) 
print(Model2_data[168:170])
print(Model2_labels[168:170])

[['order', 'one', 'extra', 'cheese', 'and', 'pepper', 'pizza', 'without', 'pesto'], ['i', 'll', 'try', 'one', 'small', 'pie', 'along', 'with', 'onion', 'mushroom', 'but', 'hold', 'pesto']]
[[2, 0, 0, 0, 2, 0, 2, 2, 0], [2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0]]
[['order', 'one', 'extra', 'cheese', 'and', 'pepper', 'pizza', 'without', 'pesto'], ['i', 'll', 'try', 'one', 'small', 'pie', 'along', 'with', 'onion', 'mushroom', 'but', 'hold', 'pesto']]
[[0, 1, 5, 4, 0, 4, 0, 0, 6], [0, 0, 0, 1, 2, 0, 0, 0, 4, 4, 0, 0, 6]]


In [53]:

path = (
    'train_data_order_details.txt' if datatype == 'train' else
    'dev_data_order_details.txt'
)

with open(path, 'a') as f: 
    for item in Model2_data:
        f.write("%s\n" % item)

In [54]:
path = (
    'train_labels_order_details.txt' if datatype == 'train' else
    'dev_labels_order_details.txt'
)
for labels in Model2_labels:
    with open(path, 'a') as f:
        f.write("%s\n" % labels)